In [1]:
!pip install --upgrade accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 2.3 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.29.3
    Uninstalling accelerate-0.29.3:
      Successfully uninstalled accelerate-0.29.3


In [2]:
!pip install transformers[torch]

In [3]:
import pandas as pd
from transformers import BartTokenizer
import torch
from transformers import BartForConditionalGeneration, Trainer, TrainingArguments
from accelerate import Accelerator, DataLoaderConfiguration



2024-05-30 06:51:42.358018: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-30 06:51:42.358119: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-30 06:51:42.618777: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
import torch

if torch.cuda.is_available():
  print("CUDA is available!")
else:
  print("CUDA is not available.")


CUDA is available!


In [5]:
# Load your data
data = pd.read_csv('/kaggle/input/data-train/data_train.csv', header=None)


In [6]:
data = data
# Create empty lists to store reactants and products
reactants = []
products = []

# Iterate through each row in column one (assuming SMILES strings are there)
for row in data.iloc[:, 0]:  # Access all rows in column 0 (index 0)
    if pd.isna(row):  # Skip rows with missing values (NaN)
        continue

    try:
        # Split the SMILES string based on ">>" (assuming one product)
        split_data = row.split('>>')

        # Extract reactants
        reactants.append(split_data[0].strip())  # Remove leading/trailing whitespace

        # Extract product
        products.append(split_data[1].strip())  # Remove leading/trailing whitespace
    except IndexError:  # Handle cases where there's no ">>" separator
        reactants.append(row.strip())  # Assume the entire row is a single reactant

# Create a new DataFrame
df = pd.DataFrame({'Reactant': reactants, 'Product': products})

In [7]:
reactants = df['Reactant']
products = df['Product']

In [8]:
# Swap the input and target data
input_data = products
target_data = reactants

In [9]:
# Initialize the tokenizer Bart
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')



tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

In [10]:
# Tokenize the new input and target data
input_encodings = tokenizer(input_data.tolist(), truncation=True, padding=True, max_length=512)
target_encodings = tokenizer(target_data.tolist(), truncation=True, padding=True, max_length=512)




In [11]:
class ChemicalDataset(torch.utils.data.Dataset):
    def __init__(self, input_encodings, target_encodings):
        self.input_encodings = input_encodings
        self.target_encodings = target_encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.input_encodings.items()}
        item['labels'] = torch.tensor(self.target_encodings['input_ids'][idx])
        return item

    def __len__(self):
        return len(self.input_encodings.input_ids)

In [12]:
# Create the dataset
dataset = ChemicalDataset(input_encodings, target_encodings)

In [13]:
# Split the dataset into train and eval sets
train_size = int(0.9 * len(dataset))
eval_size = len(dataset) - train_size
train_dataset, eval_dataset = torch.utils.data.random_split(dataset, [train_size, eval_size])

In [14]:
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')

pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [15]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    evaluation_strategy='epoch',  # Evaluate after 
    save_strategy='no',  # Save the model after each epoch
)

# Initialize the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [16]:
# Train the model key: XXccc9411173297dda42dcf9cce4729ca98bc402
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,0.123200,0.081275
2,0.060400,0.053579
3,0.040900,0.045879


TrainOutput(global_step=4377, training_loss=0.387703737167836, metrics={'train_runtime': 6339.1657, 'train_samples_per_second': 11.043, 'train_steps_per_second': 0.69, 'total_flos': 2.207472216109056e+16, 'train_loss': 0.387703737167836, 'epoch': 3.0})

In [17]:
# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Evaluation results: {'eval_loss': 0.045879025012254715, 'eval_runtime': 64.9847, 'eval_samples_per_second': 39.902, 'eval_steps_per_second': 0.631, 'epoch': 3.0}


In [43]:
print(model.device)

cuda:0


In [19]:
# Load your data
import pandas as pd
data_to_test = pd.read_csv('/kaggle/input/data-train/product_smiles_test.csv', header=None)


In [20]:
subset_data_1 = data_to_test.iloc[0:2000]
subset_data_2 = data_to_test.iloc[2001:4000]
subset_data_3 = data_to_test.iloc[4001:6000]
subset_data_4 = data_to_test.iloc[6001:7690]


In [54]:
data_to_test = subset_data_4


In [55]:
len(data_to_test)

1689

In [60]:
data_to_test.head(5)

,0
6001,CC(C)(C)[Si](C)(C)OCCC(CO[Si](C)(C)C(C)(C)C)OS...
6002,CCOC(=O)C(C)(C)N1CCC(NCc2ccc(-c3ccc(C(F)(F)F)c...
6003,CC(C)(C)CC1NC(C(=O)Nc2ccc3c(c2)C(=O)NC3=O)C(c2...
6004,Cc1ccc(NC(=O)c2ccc(CN3CCOCC3)c(Br)c2)cc1Nc1ncc...
6005,O=C(NCc1ccc(F)c(F)c1)C(COC(F)F)N(Cc1ccccc1)Cc1...


In [56]:
from tqdm import tqdm

def predict_with_beam_search(model, tokenizer, product, num_beams=25, max_length=2048):
    """
    Generates predictions for a given product using beam search.

    Args:
        model: The trained model for generation.
        tokenizer: The tokenizer used for processing text.
        product: The product SMILES string for which to predict reactants.
        num_beams: Number of beams to use in beam search (default: 25).
        max_length: Maximum length of the generated sequence (default: 2048).

    Returns:
        A list of predicted reactant SMILES strings (one for each beam).
    """
    product_encoding = tokenizer(product, truncation=True, padding=True, max_length=512, return_tensors="pt")
    product_encoding = {k: v.to(model.device) for k, v in product_encoding.items()}

    # Generate predictions using beam search
    beam_outputs = model.generate(
        **product_encoding,
        num_beams=num_beams,
        early_stopping=True,
        max_length=max_length
    )

    # Decode beam outputs and return as list
    predicted_reactants = [tokenizer.decode(out, skip_special_tokens=True) for out in beam_outputs]
    return predicted_reactants

# Create an empty list to store predicted reactants
predicted_reactants_list = []

# Progress bar with tqdm
with tqdm(total=len(data_to_test[0]), desc="Processing Products") as progress_bar:
    for product in data_to_test[0]:
        # Generate predictions using beam search function
        predicted_reactants = predict_with_beam_search(model, tokenizer, product)
        predicted_reactants_list.append(predicted_reactants)  # List of predictions for each product

        # Update progress bar after each iteration
        progress_bar.update(1)

# Create a pandas DataFrame from the predicted reactants
output_df = pd.DataFrame({
    'product': data_to_test[0].tolist(),  # Assuming data_to_test[0] contains product SMILES
    'predicted_reactants': predicted_reactants_list  # List of lists (one inner list per product)
})

Processing Products: 100%|██████████| 1689/1689 [26:35<00:00,  1.06it/s]


In [57]:
output = output_df["predicted_reactants"]
output.head(10)

0    [C[Si](C)(C)OC[C@H](CCCO)[C@@H](C)C.CS(=O)(=O)Cl]
1    [CCOC(=O)C(C)(C)N1CCC(N)CC1.O=Cc1ccc(-c2ccc(C(...
2    [CC(C)(C)CC1NC(C(=O)O)C(c2cccc(Cl)c2F)C1(C#N)c...
3    [C1COCCN1.Cc1ccc(Nc2nccc(-c3cccnc3)n2)cc1Nc1nc...
4             [Nc1ccc(CNC(=O)C(COC(F)F)Cc2ccccc2)cc1F]
5    [CC(C)(C)[C@@H](CO)Nc1cc(-c2n[nH]c3cc(F)cnc3c2...
6    [Cc1c(SCC2CCN(C(=O)Cc3ccc(Cl)nc3)CC2)ccc2c1COC...
7    [C=CC(=O)Nc1cccc(-c2cccc3cnc(Nc4ccc(Br)cc4)nc2...
8    [O=C(Nc1ccc(F)cc1)[C@H]1C[C@@H](Cc2ccccc2F)CN1...
9             [BrCc1ccc(C(F)(F)F)cc1.COC(=O)c1ncccc1N]
Name: predicted_reactants, dtype: object

In [58]:
len(output)

1689

In [59]:
# Save the DataFrame as a CSV file
output_df.to_csv('predicted_reactants_fourth.csv', index=False)  # Adjust filename as needed

print(f"Predictions saved to 'predicted_reactants.csv'")

Predictions saved to 'predicted_reactants.csv'
